<a href="https://colab.research.google.com/github/renweizhukov/jupyter-lab-notebook/blob/main/hugging-face-agents-course/multiagent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [2]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 43.0 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [7]:
import os
from google.colab import userdata
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, LiteLLMModel, VisitWebpageTool


#model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
#model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto")
model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-lite",
    api_key=userdata.get("GEMINI_API_TOKEN")
)

We can start with creating a baseline, simple agent to give us a simple report.

In [17]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [9]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [38]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [39]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  from geopy.geocoders import Nominatim                                                                            
  from geopy.exc import GeocoderTimedOut                                                                           
                                                                                                                   
  def get_coordinates(city_name, country_name=""):                                                                 
      geolocator = Nominatim(user_agent="geo_locator")                                                             
      try:                                                                                                         
          if country_name:                                                                                         
              location = geolocator.geocode(f"{city_name}, {country_name}")                                        
          else:                                                                                                    
              location = geolocator.geocode(city_name)                                                             
          if location:                                                                                             
              return (location.latitude, location.longitude)                                                       
          else:                                                                                                    
              return None                                                                                          
      except GeocoderTimedOut:                                                                                     
          print(f"GeocoderTimedOut for {city_name}")                                                               
          return None                                                                                              
      except Exception as e:                                                                                       
          print(f"Error geocoding {city_name}: {e}")                                                               
          return None                                                                                              
                                                                                                                   
  batman_locations_search = web_search(query="Batman filming locations")                                           
  print(batman_locations_search)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'from geopy.geocoders import Nominatim' due to: InterpreterError: Import from 
geopy.geocoders is not allowed. Authorized imports are: ['datetime', 'collections', 'random', 're', 'stat', 
'queue', 'time', 'statistics', 'math', 'itertools', 'pandas', 'unicodedata'\]

[Step 1: Duration 5.60 seconds| Input tokens: 2,485 | Output tokens: 317]

...

[Step 10: Duration 3.98 seconds| Input tokens: 331,629 | Output tokens: 9,388]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(f"Batman Filming Locations:\n{batman_df.to_string()}\n\nSupercar Factory                            
  Locations:\n{supercar_df.to_string()}")                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Batman Filming Locations:
              location  travel_time_hours
0          Glasgow, UK               8.60
1        Liverpool, UK               8.81
2           London, UK               9.17
3        Knebworth, UK               9.15
4         Chicago, USA               2.68
5     Los Angeles, USA               6.77
6  Buckinghamshire, UK               9.09
7            Essex, UK               9.23

Supercar Factory Locations:
                                    factory  travel_time_hours
0                 Ferrari, Maranello, Italy              10.72
1                       McLaren, Woking, UK               9.14
2                  Aston Martin, Gaydon, UK               9.01
3  Lamborghini, Sant'Agata Bolognese, Italy              10.74

[Step 11: Duration 1.65 seconds| Input tokens: 409,635 | Output tokens: 9,473]

In [40]:
result

"Batman Filming Locations:\n              location  travel_time_hours\n0          Glasgow, UK               8.60\n1        Liverpool, UK               8.81\n2           London, UK               9.17\n3        Knebworth, UK               9.15\n4         Chicago, USA               2.68\n5     Los Angeles, USA               6.77\n6  Buckinghamshire, UK               9.09\n7            Essex, UK               9.23\n\nSupercar Factory Locations:\n                                    factory  travel_time_hours\n0                 Ferrari, Maranello, Italy              10.72\n1                       McLaren, Woking, UK               9.14\n2                  Aston Martin, Gaydon, UK               9.01\n3  Lamborghini, Sant'Agata Bolognese, Italy              10.74"

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [87]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
*   Gotham's coordinates: 40.7128° N, 74.0060° W. This is the origin for cargo plane travel time calculations.
*   The task specifies finding Batman filming locations.
*   The task specifies finding supercar factories.
*   The task requests a pandas DataFrame as the output.
*   The task specifies calculating cargo plane travel time.
*   The task implicitly suggests the use of a cargo plane.

### 1.2. Facts to look up
*   Batman filming locations (specific locations and their coordinates): Need to use web search to find the filming
locations. Sources will be websites detailing film locations.
*   Supercar factories (locations and their coordinates): Need to use web search to find supercar factories. 
Sources will be websites listing factories.
*   Confirmation of information on websites.

### 1.3. Facts to derive
*   Cargo plane travel time from Gotham to each Batman filming location (using the given coordinates and the 
`calculate_cargo_travel_time` tool).
*   Cargo plane travel time from Gotham to each supercar factory.
*   Comparison of travel times to match Batman filming locations with supercar factory locations that have 
approximately the same travel time.
*   Dataframe with relevant data, including: Location name, Location coordinates (latitude, longitude), Travel time
from Gotham for both Batman filming locations and supercar factories.

## 2. Plan

1.  **Search for Batman filming locations:** Use a web search to find a list of Batman filming locations.
2.  **Iterate through Batman filming locations and collect coordinates**: For each location found in the search 
results, visit the corresponding webpage (or multiple, if necessary) to confirm and extract the location's 
coordinates (latitude, longitude).
3.  **Calculate cargo plane travel time to each Batman location:** Using the extracted coordinates of each Batman 
location and the coordinates of Gotham (40.7128° N, 74.0060° W), calculate the cargo plane travel time using the 
`calculate_cargo_travel_time` tool. Store the travel time and the location details.
4.  **Search for Supercar factories:** Use web search to find a list of Supercar factories.
5.  **Iterate through Supercar factories and collect coordinates**: For each factory found, visit the corresponding
webpage (or multiple, if necessary) to confirm and extract the factory's coordinates (latitude, longitude).
6.  **Calculate cargo plane travel time to each Supercar factory:** Using the extracted coordinates of each factory
and the coordinates of Gotham (40.7128° N, 74.0060° W), calculate the cargo plane travel time using the 
`calculate_cargo_travel_time` tool. Store the travel time and factory details.
7.  **Create a pandas DataFrame for Batman filming locations**: Create a DataFrame with columns for location name, 
coordinates (latitude, longitude), and travel time from Gotham.
8.  **Create a pandas DataFrame for Supercar factories**: Create a DataFrame with columns for factory name, 
coordinates (latitude, longitude), and travel time from Gotham.
9.  **Compare and match travel times**: For each Batman location, identify a supercar factory with a similar travel
time (within a reasonable margin of error).
10. **Populate Dataframe**: For each match, populate both the Batman and Supercar DataFrames.
11. **Return the DataFrames**: Return the filled pandas DataFrames.
12. **Final answer**: Present the combined data of Batman locations and matching supercar factories with the 
respective travel times.
\n
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  locations_search = web_search(query="Batman filming locations", filter_year=None)                                
  print(locations_search)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoouy-mDEXsdk0nrAcakKGEuNDrnIR_sFYlIi7r
OfbuSqnuBVonV)
Date published: Apr 20, 2023
Source: LA Today

Los Angeles, CA. The legendary Batcave is found in Bronson Canyon, at the Bronson Caves. This is a section of the 
expansive Griffith Park in Los ...

6. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

7. [The Batman movie filming location // Thompson Center 
...](https://www.reddit.com/r/batman/comments/jeqdei/the_batman_movie_filming_location_thompson_center/)
Source: Reddit · r/batman

The Batman movie filming location // Thompson Center Downtown Chicago · Comments Section · Community Info Section ·
More posts you may like. One ...

8. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Coordinates and travel times for Batman filming locations from previous steps                                  
  batman_locations = [                                                                                             
      {"name": "St George's Hall, Liverpool, England, UK", "latitude": 53.4086, "longitude": -2.9801,              
  "travel_time_hours": 8.81},                                                                                      
      {"name": "Royal Liver Building, Liverpool, England, UK", "latitude": 53.4035, "longitude": -2.9908,          
  "travel_time_hours": 8.81},                                                                                      
      {"name": "Knebworth House, Knebworth, Hertfordshire, England, UK", "latitude": 51.7718, "longitude":         
  -0.2006, "travel_time_hours": 9.15},                                                                             
      {"name": "James R Thompson Center, Chicago, USA", "latitude": 41.8853, "longitude": -87.6322,                
  "travel_time_hours": 2.68},                                                                                      
      {"name": "Bronson Canyon, Griffith Park, Los Angeles, USA", "latitude": 34.1244, "longitude": -118.3142,     
  "travel_time_hours": 6.78}                                                                                       
  ]                                                                                                                
                                                                                                                   
  # Supercar factories with coordinates and calculated travel times                                                
  supercar_factories = [                                                                                           
      {"factory_name": "Ferrari Factory, Maranello, Italy", "latitude": 44.5324, "longitude": 10.8641,             
  "travel_time_hours": 10.74},  # Coordinates found                                                                
      {"factory_name": "Lamborghini Factory, Sant'Agata Bolognese, Italy", "latitude": 44.658933, "longitude":     
  11.124913, "travel_time_hours": 10.74},  # Coordinates found                                                     
      {"factory_name": "Pagani Automobili, San Cesario sul Panaro, Italy", "latitude": 44.562105, "longitude":     
  11.034807, "travel_time_hours": 10.74},                                                                          
      {"factory_name": "SSC North America, Richland, Washington, USA", "coordinates": None[38;2;248;248;242;48;2;3

Out - Final answer:                                        location_name  latitude  longitude  \
0           St George's Hall, Liverpool, England, UK   53.4086    -2.9801   
1       Royal Liver Building, Liverpool, England, UK   53.4035    -2.9908   
2  Knebworth House, Knebworth, Hertfordshire, Eng...   51.7718    -0.2006   
3              James R Thompson Center, Chicago, USA   41.8853   -87.6322   
4    Bronson Canyon, Griffith Park, Los Angeles, USA   34.1244  -118.3142   

   travel_time_hours                      factory_match  
0               8.81  Ferrari Factory, Maranello, Italy  
1               8.81  Ferrari Factory, Maranello, Italy  
2               9.15  Ferrari Factory, Maranello, Italy  
3               2.68  Ferrari Factory, Maranello, Italy  
4               6.78  Ferrari Factory, Maranello, Italy  

[Step 7: Duration 7.93 seconds| Input tokens: 83,475 | Output tokens: 3,808]

                                       location_name  latitude  longitude  \
0           St George's Hall, Liverpool, England, UK   53.4086    -2.9801   
1       Royal Liver Building, Liverpool, England, UK   53.4035    -2.9908   
2  Knebworth House, Knebworth, Hertfordshire, Eng...   51.7718    -0.2006   
3              James R Thompson Center, Chicago, USA   41.8853   -87.6322   
4    Bronson Canyon, Griffith Park, Los Angeles, USA   34.1244  -118.3142   

   travel_time_hours                      factory_match  
0               8.81  Ferrari Factory, Maranello, Italy  
1               8.81  Ferrari Factory, Maranello, Italy  
2               9.15  Ferrari Factory, Maranello, Italy  
3               2.68  Ferrari Factory, Maranello, Italy  
4               6.78  Ferrari Factory, Maranello, Italy  


In [88]:
detailed_report

,location_name,latitude,longitude,travel_time_hours,factory_match
0,"St George's Hall, Liverpool, England, UK",53.4086,-2.9801,8.81,"Ferrari Factory, Maranello, Italy"
1,"Royal Liver Building, Liverpool, England, UK",53.4035,-2.9908,8.81,"Ferrari Factory, Maranello, Italy"
2,"Knebworth House, Knebworth, Hertfordshire, Eng...",51.7718,-0.2006,9.15,"Ferrari Factory, Maranello, Italy"
3,"James R Thompson Center, Chicago, USA",41.8853,-87.6322,2.68,"Ferrari Factory, Maranello, Italy"
4,"Bronson Canyon, Griffith Park, Los Angeles, USA",34.1244,-118.3142,6.78,"Ferrari Factory, Maranello, Italy"


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [10]:
#model = HfApiModel(
#    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
#)
model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-lite",
    api_key=userdata.get("GEMINI_API_TOKEN")
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serpapi"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [11]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [12]:
!pip install google-generativeai smolagents

In [13]:
import os
import google.generativeai as genai

class GeminiProModel:
    def __init__(self, model="gemini-1.5-flash", max_tokens=8192, temperature=0.7):
        genai.configure(api_key=userdata.get('GEMINI_API_TOKEN'))
        self._model = genai.GenerativeModel(model)
        self.model_id = model
        self.max_tokens = max_tokens
        self.temperature = temperature

    def generate(self, prompt, **kwargs):
        # Convert chat-style messages to plain string if needed
        if isinstance(prompt, list):
            # Format list of messages: [{'role': 'user', 'content': '...'}]
            prompt_text = "\n".join(f"{m['role']}: {m['content']}" for m in prompt)
        else:
            prompt_text = prompt

        response = self._model.generate_content(
            prompt_text,
            generation_config={
                "max_output_tokens": self.max_tokens,
                "temperature": self.temperature,
            }
        )

        # Return an object with `.content`, as expected
        return type("Response", (), {"content": response.text})

In [14]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    #model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    model=GeminiProModel(model="gemini-1.5-flash", max_tokens=8192),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "csv",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [15]:
manager_agent.visualize()

CodeAgent | gemini-1.5-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'csv', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gemini/gemini-2.0-flash-lite
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.        

In [98]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ GeminiProModel - gemini-1.5-flash ─────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task

* Gotham coordinates: 40.7128° N, 74.0060° W.
* Requirement to find at least 6 locations (Batman filming locations + supercar factories).
* Output format: Spatial map saved as `saved_map.png`, using scatter points colored by travel time.
* Available tools: `calculate_cargo_travel_time`, `final_answer`, `web_agent`.


### 1.2. Facts to look up

* **Batman filming locations:**  Need to find a comprehensive list of locations worldwide where Batman movies or TV
series have been filmed.  Possible sources: IMDb, Wikipedia (Batman filming locations pages), dedicated fan 
websites, news articles about filming locations.
* **Supercar factory locations:** Need a list of supercar manufacturers and their factory locations. Possible 
sources: Manufacturer websites, automotive news sites, Wikipedia pages for each manufacturer.


### 1.3. Facts to derive

* **Travel time to each location:** For each Batman filming location and supercar factory, calculate the travel 
time from Gotham using `calculate_cargo_travel_time`.  This requires the latitude and longitude coordinates for 
each location.
* **Spatial map data:** Create a structured dataset (e.g., a Pandas DataFrame) with location names, latitude, 
longitude, and travel time to be used for plotting the map.


## 2. Plan

1. **Gather Batman filming locations:**  Use `web_agent("Find all worldwide filming locations for Batman movies and
TV series.")` to retrieve a list of locations.  Parse the response to extract location names and coordinates 
(latitude and longitude).  If the response doesn't directly provide coordinates, use `web_agent` again with 
specific location names to retrieve their coordinates.

2. **Gather supercar factory locations:** Use `web_agent("Find the locations of major supercar factories worldwide 
(include latitude and longitude).")` to retrieve a list of factories and their coordinates.

3. **Calculate travel times:** For each location (Batman filming locations and supercar factories) obtained in 
steps 1 and 2, use `calculate_cargo_travel_time` to calculate the travel time from Gotham (40.7128° N, 74.0060° W).
Assume a standard cruising speed for cargo planes (the default in the function).

4. **Create dataset for plotting:** Create a dataset (e.g., a Pandas DataFrame) with columns for location name, 
latitude, longitude, and calculated travel time.  This dataset should contain at least 6 entries (Batman locations 
+ supercar factories).

5. **Generate and save the map:** Use `plotly.express` to generate a scatter map using the dataset created in step 
4.  The color of each point should represent the travel time. Save the map as `saved_map.png`.

6. **Return the final answer:** Use `final_answer(fig)` to return the generated map.

<end_plan>

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
MessageRole.ASSISTANT: [{'type': 'text', 'text': 'Thought: I will start by gathering Batman filming locations using
the web_agent tool.  I will then do the same for supercar factories.  The results will be printed and used in the  
next step.\n\nCode:\n```python\nbatman_locations_raw = web_agent("Find all worldwide filming locations for Batman  
movies and TV series.  Include latitude and longitude if possible.")\nprint(f"Raw Batman locations:                
{batman_locations_raw}")\nsupercar_factories_raw = web_agent("Find the locations of major supercar factories       
worldwide (include latitude and longitude).")\nprint(f"Raw supercar factories:                                     
{supercar_factories_raw}")\n```<end_code>\n\nObservation:\nRaw Batman locations:  (This will contain the output    
from the web agent.  I cannot execute the web_agent function here.)\nRaw supercar factories: (This will contain the
output from the web agent. I cannot execute the web_agent function here.)\n\nThought: The web agent responses are  
likely to be strings containing lists of locations.  I need to process these strings to extract location names and 
coordinates.  Since I cannot process strings directly, I will need a human to help. I will ask the team member to  
extract the relevant information and return it as a structured data. Then I will calculate travel                  
times.\n\nCode:\n```python\nteam_member_task = f[38;2;230;237;243;48;2;1

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
Here is your code snippet:
MessageRole.ASSISTANT: [{'type': 'text', 'text': 'Thought: I will start by gathering Batman filming locations using
the web_agent tool.  I will then do the same for supercar factories.  The results will be printed and used in the 
next step.\n\nCode:\n```python\nbatman_locations_raw = web_agent("Find all worldwide filming locations for Batman 
movies and TV series.  Include latitude and longitude if possible.")\nprint(f"Raw Batman locations: 
{batman_locations_raw}")\nsupercar_factories_raw = web_agent("Find the locations of major supercar factories 
worldwide (include latitude and longitude).")\nprint(f"Raw supercar factories: 
{supercar_factories_raw}")\n```<end_code>\n\nObservation:\nRaw Batman locations:  (This will contain the output 
from the web agent.  I cannot execute the web_agent function here.)\nRaw supercar factories: (This will contain the
output from the web agent. I cannot execute the web_agent function here.)\n\nThought: The web agent responses are 
likely to be strings containing lists of locations.  I need to process these strings to extract location names and 
coordinates.  Since I cannot process strings directly, I will need a human to help. I will ask the team member to 
extract the relevant information and return it as a structured data. Then I will calculate travel 
times.\n\nCode:\n```python\nteam_member_task = f\"The following text contains information about Batman filming 
locations and supercar factories. Please extract the location names and their latitude and longitude coordinates. 
Return the data in a format suitable for creating a Pandas DataFrame, for instance a list of dictionaries, where 
each dictionary contains 'location', 'latitude', and 'longitude'.\\n\\nBatman locations: 
{batman_locations_raw}\\n\\nSupercar factories: {supercar_factories_raw}\"\nprocessed_locations = 
web_agent(task=team_member_task)\nprint(f\"Processed locations: 
{processed_locations}\")\n```<end_code>\n\nObservation:\nProcessed locations: (This will be a structured list of 
dictionaries containing location information, provided by the human team member.)\n\nThought: Now that I have 
processed location data, I will create a Pandas DataFrame and calculate travel times using 
`calculate_cargo_travel_time`. Gotham coordinates are 40.7128° N, 74.0060° W.\n\nCode:\n```python\nimport pandas as
pd\n\ngothem_coords = (40.7128, -74.0060)\ndata = eval(processed_locations) # Assuming processed_locations is a 
string representation of a list of dictionaries\ndf = pd.DataFrame(data)\ndf[\"travel_time\"\] = df.apply(lambda 
row: calculate_cargo_travel_time(gothem_coords, (row[\"latitude\"\], row[\"longitude\"\]), 750), 
axis=1)\nprint(df)\n```<end_code>\n\nObservation:\n(This will show the Pandas DataFrame with location names, 
latitude, longitude, and calculated travel times.)\n\nThought: Finally, I will generate the map using Plotly 
Express and save it.\n\nCode:\n```python\nimport plotly.express as px\n\nfig = px.scatter_mapbox(df, 
lat=\"latitude\", lon=\"longitude\", text=\"location\", color=\"travel_time\",\n                        size=100, 
color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1,\n                        height=600, 
width=800, mapbox_style=\"carto-positron\")\nfig.write_image(\"saved_map.png\")\nfinal_answer(fig)\n```<end_code>}]

It seems like you're trying to return the final answer, you can do it as follows:
Code:
```py
final_answer("YOUR FINAL ANSWER HERE")
```<end_code>
Make sure to provide correct code blobs.

...

Check check_reasoning_and_plot failed with error: Make sure to save the plot under saved_map.png!

[Step 15: Duration 45.32 seconds]

Reached max steps.

[Step 16: Duration 45.33 seconds]

"Error in generating final LLM output:\n'GeminiProModel' object is not callable"

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [1]:
manager_agent.python_executor.state["fig"]

NameError: name 'manager_agent' is not defined

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)